# Fetch historical data

Python script to fetch historical data from binance using ccxt

In [1]:
# Install openpyxl and CCXT
!pip install openpyxl ccxt

  Using cached yarl-1.1.0.tar.gz (156 kB)
  Using cached aiodns-2.0.0-py2.py3-none-any.whl (4.8 kB)
  Using cached aiohttp-3.7.3-cp38-cp38-win_amd64.whl (634 kB)
  Using cached async_timeout-3.0.1-py3-none-any.whl (8.2 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached cryptography-3.3.1-cp36-abi3-win_amd64.whl (1.5 MB)
  Using cached multidict-5.1.0-cp38-cp38-win_amd64.whl (48 kB)
  Using cached pycares-3.1.1-cp38-cp38-win_amd64.whl (70 kB)
  Created wheel for yarl: filename=yarl-1.1.0-cp38-cp38-win_amd64.whl size=121260 sha256=d4fbb4a70c3cba264e5c0cfa7f08219ac6769bae228890eab2aa40438ca28c7e
  Stored in directory: c:\users\kit\appdata\local\pip\cache\wheels\82\ad\ec\51f1c7833ed931c471836a6f1afc7c2664bd898e0b4ce30d25
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8913 sha256=1116b68c69e2d5a8ee20ddde16b27cc380eac837e405f722095f4330462b23c3
  Stored in directory: c:\users\kit\appdata\local\pip\cache\wheels\6e\df\38\abda47b884e3e

In [2]:
import os
from pathlib import Path

import sys
import csv

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)


In [3]:
scrape_candles_to_csv('btc_usdt_1m.csv', 'binance', 3, 'BTC/USDT', '1m', '2017-01-0100:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/eth_btc_1m.csv', 'binance', 3, 'ETH/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/ltc_btc_1m.csv', 'binance', 3, 'LTC/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/xlm_btc_1m.csv', 'binance', 3, 'XLM/BTC', '1m', '2018-01-01T00:00:00Z', 1000)

1000 BTC/USDT candles in total from 2021-01-03T16:31:00.000Z to 2021-01-04T09:10:00.000Z
2000 BTC/USDT candles in total from 2021-01-02T23:51:00.000Z to 2021-01-04T09:10:00.000Z
3000 BTC/USDT candles in total from 2021-01-02T07:11:00.000Z to 2021-01-04T09:10:00.000Z
4000 BTC/USDT candles in total from 2021-01-01T14:31:00.000Z to 2021-01-04T09:10:00.000Z
5000 BTC/USDT candles in total from 2020-12-31T21:51:00.000Z to 2021-01-04T09:10:00.000Z
6000 BTC/USDT candles in total from 2020-12-31T05:11:00.000Z to 2021-01-04T09:10:00.000Z
7000 BTC/USDT candles in total from 2020-12-30T12:31:00.000Z to 2021-01-04T09:10:00.000Z
8000 BTC/USDT candles in total from 2020-12-29T19:51:00.000Z to 2021-01-04T09:10:00.000Z
9000 BTC/USDT candles in total from 2020-12-29T03:11:00.000Z to 2021-01-04T09:10:00.000Z
10000 BTC/USDT candles in total from 2020-12-28T10:31:00.000Z to 2021-01-04T09:10:00.000Z
11000 BTC/USDT candles in total from 2020-12-27T17:51:00.000Z to 2021-01-04T09:10:00.000Z
12000 BTC/USDT cand

KeyboardInterrupt: 